In [ ]:
# multivariate multi-step stacked lstm example
import numpy as np
from numpy import array
from numpy import hstack
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Bidirectional
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import pandas as pd
import pickle as pkl
from sklearn.metrics import mean_squared_error
import tensorflow as tf

In [ ]:
nthu_train_X = pkl.load(open("../input/ddd-data-preparation-nthu/nthu_train_X", 'rb'))
nthu_train_y = pkl.load(open("../input/ddd-data-preparation-nthu/nthu_train_y", 'rb'))

nthu_test_X = pkl.load(open("../input/ddd-data-preparation-nthu/nthu_test_X", 'rb'))
nthu_test_y = pkl.load(open("../input/ddd-data-preparation-nthu/nthu_test_y", 'rb'))

In [ ]:
nthu_train_y = nthu_train_y.mean(axis=1)
nthu_test_y = nthu_test_y.mean(axis=1)

In [ ]:
nthu_train_y_scaled = tf.keras.activations.sigmoid(tf.constant(nthu_train_y, dtype = tf.float32))
nthu_train_y_scaled = nthu_train_y_scaled.numpy()

In [ ]:
nthu_test_y_scaled = tf.keras.activations.sigmoid(tf.constant(nthu_test_y, dtype = tf.float32))
nthu_test_y_scaled = nthu_test_y_scaled.numpy()

In [ ]:
n_steps_in, n_features, n_steps_out= 150, 8, 1

In [ ]:
nthu_train_X_reshaped = nthu_train_X.reshape((-1, 3, 50, n_features))

In [ ]:
idx =[]
np.random.seed(0)
for i in range(10000):
    idx.append(np.random.randint(420953))

In [ ]:
# define model
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=128, kernel_size=1, activation='tanh'), input_shape=(None,50, n_features)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(50, activation='tanh'))
model.add(Dense(n_steps_out, activation='sigmoid'))
model.compile(optimizer='adam', loss='mse')
# fit model
model.fit(nthu_train_X_reshaped[idx], np.ravel(nthu_train_y_scaled)[idx], batch_size=8, epochs=4, verbose=2)

In [ ]:
nthu_test_X = nthu_test_X.reshape((-1, 3, 50, n_features))

In [ ]:
nthu_test_X.shape

In [ ]:
# demonstrate prediction
yhat = model.predict(nthu_test_X[:1000], verbose=1)
print(yhat)

In [ ]:
def logit(x):
    """ Computes the logit function, i.e. the logistic sigmoid inverse. """
    if (x<=0.5):
        return float(0)
    if (x>=0.880797):
        return float(2)
    return float(- tf.math.log(1. / x - 1.))

In [ ]:
yhat_inversed = np.array([logit(x) for x in yhat])

In [ ]:
mean_squared_error(yhat_inversed,nthu_test_y)

In [ ]:
model.save("lstm_cnn.h5")